In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics
import xgboost as xgb
import os
import time
import pickle
color = sns.color_palette()
%matplotlib inline


In [7]:
# constants
DATA_PATH = '/kaggle/dev/ashish/imba/data'

In [11]:
lgbm_preds = pickle.load(open(os.path.join(DATA_PATH, "prediction_lgbm.pkl"), "rb"))
xgb_preds = pickle.load(open(os.path.join(DATA_PATH, "prediction_xgb.pkl"), "rb"))

In [13]:
lgbm_preds.head()

,order_id,prediction,product_id
0,65432,0.030661,17330
1,203668,0.031806,17330
2,1734476,0.025635,17330
3,1627971,0.034814,17330
4,1213682,0.034307,17330


In [14]:
xgb_preds.head()

,order_id,prediction,product_id
0,65432,0.203147,17330
1,203668,0.207334,17330
2,1734476,0.228464,17330
3,1627971,0.212226,17330
4,1213682,0.204465,17330


In [17]:
all_preds = pd.merge(lgbm_preds, xgb_preds, on=['order_id', 'product_id'], how='outer')
all_preds.head()

,order_id,prediction_x,product_id,prediction_y
0,65432,0.030661,17330,0.203147
1,203668,0.031806,17330,0.207334
2,1734476,0.025635,17330,0.228464
3,1627971,0.034814,17330,0.212226
4,1213682,0.034307,17330,0.204465


In [26]:
weights = np.asarray([0.4032795, 0.2732466])
weights = weights / np.sum(weights)
weights

array([ 0.59610339,  0.40389661])

In [28]:
all_preds['prediction'] = (all_preds.prediction_x * weights[0] + all_preds.prediction_y * weights[1]) / 2
all_preds.head()

,order_id,prediction_x,product_id,prediction_y,prediction
0,65432,0.030661,17330,0.203147,0.050164
1,203668,0.031806,17330,0.207334,0.051351
2,1734476,0.025635,17330,0.228464,0.053779
3,1627971,0.034814,17330,0.212226,0.053235
4,1213682,0.034307,17330,0.204465,0.051517


In [29]:
all_preds.to_pickle(os.path.join(DATA_PATH, 'prediction_all_weighted.pkl'))